In [82]:
import networkx as nx
from matplotlib import pyplot as plt
import random
import math
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import threading
import queue

In [83]:
G = nx.Graph(time =3)
G.graph['S'] = 0
G.graph['I'] = 0
G.graph['R'] = 0

In [84]:
G.graph['S']

0

In [85]:
def initialize_graph(node_count, contact_radius = 1, queue = None, time = 0, edge_dist = 'geometric'):
    square_size = math.sqrt(node_count) #size of square
    custom_pos = {} #custom position for plotting
    G = nx.Graph(time = time) #initialize graph
    G.graph['S_count'] = 0 #initialize S count
    G.graph['I_count'] = 0 #initialize I count
    G.graph['R_count'] = 0 #initialize R count
    for i in range(node_count):
        G.add_node(i)
        G.nodes[i]['state'] = 'S'  
        x,y = random.uniform(0, square_size), random.uniform(0, square_size)
        G.nodes[i]['pos'] = (x,y)
        G.nodes[i]['observed'] = False
        G.nodes[i]['SIR_prob'] = [1,0,0]
        G.nodes[i]['infected_time'] = 0
        G.graph['S_count'] += 1
        custom_pos[i] = (x,y)
    #initialize node state, color, and position
    #set the colors
    #S = susceptible (green)
    #I = infected (red)
    #R = recovered (blue)
    #set the edges
    edges = []
    if edge_dist == 'geometric':
        edges = nx.geometric_edges(G, radius = contact_radius)
    elif edge_dist == 'exp':
        p_dist = lambda distance: math.exp(-distance/contact_radius)
        edges = nx.soft_random_geometric_graph(node_count, radius = contact_radius, pos=custom_pos, p = 2, p_dist=p_dist).edges
    G.add_edges_from(edges)
    queue.put((G, custom_pos))

In [86]:
def infectRandomNode(G, node_count):
    #infect a random node that is susceptible
    sample = random.sample(G.nodes(), G.number_of_nodes())
    count = 0
    for i in sample:
        if (count >= node_count):
            break
        if G.nodes[i]['state'] == 'S':
            G.nodes[i]['state'] = 'I'
            G.graph['I_count'] += 1
            count += 1

In [87]:
def update(G_t1, G_t2, infection_rate, recovery_rate):
    #update Gt2 based on G_t1 assuming they are similar node size
    #copy graph data:
    #copy SIR counts
    G_t2.graph['S_count'] = G_t1.graph['S_count']
    G_t2.graph['I_count'] = G_t1.graph['I_count']
    G_t2.graph['R_count'] = G_t1.graph['R_count']
    for node in G_t1.nodes():
        G_t2.nodes[node]['observed'] = G_t1.nodes[node]['observed']
        #check node state
        if G_t1.nodes[node]['state'] == 'S':
            for neighbor in G_t1.neighbors(node):
                if G_t1.nodes[neighbor]['state'] == 'I':
                    if random.uniform(0, 1) < infection_rate:
                        G_t2.nodes[node]['state'] = 'I'
                        G_t2.graph['I_count'] += 1
                        break
        elif G_t1.nodes[node]['state'] == 'I':
            if random.uniform(0, 1) < recovery_rate:
                G_t2.nodes[node]['state'] = 'R'
                G_t2.graph['R_count'] += 1
            else:
                G_t2.nodes[node]['state'] = 'I'
            G_t2.nodes[node]['infected_time'] = G_t1.nodes[node]['infected_time'] + 1
        else:
            G_t2.nodes[node]['state'] = 'R'
        #copy SIR probabilities
        G_t2.nodes[node]['SIR_prob'] = G_t1.nodes[node]['SIR_prob']
    return G_t2

In [88]:
def SurroundInfectionRate(graph, node, infection_rate):
    #total infection rate of a node based on the infection rate of its neighbors
    neighbors = list(graph.neighbors(node))
    result = 0
    for neighbor in neighbors:
        result += infection_rate * graph.nodes[neighbor]['SIR_prob'][1]
    return result

In [89]:
def MeanFieldInference(graph_t, infection_rate, recovery_rate):
    predicted_state_t1 = {}
    for node in graph_t.nodes():
        factor = SurroundInfectionRate(graph_t, node, infection_rate)
        Ps = graph_t.nodes[node]['SIR_prob'][0]*(1-factor)
        Pi = (1-recovery_rate)*graph_t.nodes[node]['SIR_prob'][1] + graph_t.nodes[node]['SIR_prob'][0]*factor
        Pr = graph_t.nodes[node]['SIR_prob'][2] + recovery_rate*graph_t.nodes[node]['SIR_prob'][1]
        predicted_state_t1[node] = [Ps, Pi, Pr]
    return predicted_state_t1

In [90]:
#given temporal graph, node, and state, update the node's state probability in the past delta time steps to be the given state
def BackloopUpdate(temporal_graphs,t ,delta, node, state):
    mapper = {'S':[1,0,0], 'I':[0,1,0], 'R':[0,0,1]}
    lower_bound = max(0, t-delta)
    for i in range(lower_bound, t):
        temporal_graphs[i][0].nodes[node]['SIR_prob'] = mapper[state]
        temporal_graphs[i][0].nodes[node]['observed'] = True

In [91]:
#@param: give a list of nodes that are to be observed
#@pre: the list of nodes are ordered by rank
#@modifies: the state probability of the nodes in the past delta time steps
def DetectObserve(obs, temporal_graphs, observe_per_day, t, delta):
    #robustness check
    if (delta >= t):
        delta = t
    list_of_nodes = []
    count = 0
    for entry in obs:
        if count == observe_per_day:
            return list_of_nodes
        state = temporal_graphs[t][0].nodes[entry]['state']
        obs_state = temporal_graphs[t][0].nodes[entry]['observed']
        #check if the node is already observed if so, skip
        if obs_state == True:
            continue
        if state == 'S':
            BackloopUpdate(temporal_graphs, t, delta, entry, 'S')
        elif state == 'I':
            BackloopUpdate(temporal_graphs, t, delta, entry, 'I')
        elif state == 'R':
            BackloopUpdate(temporal_graphs, t, delta, entry, 'R')
        count += 1
        list_of_nodes.append(entry)
    return list_of_nodes
        
        

In [92]:
#given a graph, a list of nodes, set the nodes to be observed and put into removed
def SetRemove(graph, list):
    for entry in list:
        #print(entry)
        graph.nodes[entry]['observed'] = True
        graph.nodes[entry]['SIR_prob'] = [0,0,1]
        graph.nodes[entry]['state'] = 'R'

In [93]:
#sample a list of size s_count nodes that are not observed
def Sampler(graph, s_count):
    nodes = list(graph.nodes())
    random.shuffle(nodes)
    result = []
    count = 0
    for node in nodes:
        if graph.nodes[node]['observed']==False:
            result.append(node)
            count += 1
        if count == s_count:
            return result
    return result

In [94]:
#@generate nr_graphs graph and put them into temporal_graph_dict and return it
def generate_graphs(node_count, contact_radius, nr_graphs):
    que_graphs = queue.Queue()
    thread_list = []
    for t in range(0, nr_graphs):
        thread = threading.Thread(target=initialize_graph, args=(node_count, contact_radius, que_graphs, t), name='thread'+str(t))
        thread.start()
        #print('thread'+str(t)+' started')
        thread_list.append(thread)

    for thread in tqdm(thread_list, desc='Generating Graphs'):
        #print('thread'+thread.name +' joined')
        thread.join()
    temporal_graph_dict = {}
    while not que_graphs.empty():
        G,custom_pos = que_graphs.get()
        temporal_graph_dict[G.graph['time']] = (G, custom_pos)
    return temporal_graph_dict

In [95]:
def contactTrace(temporalgraph_dic, t, node, delta):
    #count the number of nodes that are infected in the past delta days
    count = 0
    for i in range(t-delta, t):
        for neighbor in temporalgraph_dic[i][0].neighbors(node):
            if temporalgraph_dic[i][0].nodes[neighbor]['observed'] == True and temporalgraph_dic[i][0].nodes[neighbor]['state'] == 'I':
                count += 1
    return count

In [96]:
def self_reporting(temporal_graph_dict, t, delta, self_report_days):
    for node in temporal_graph_dict[t][0].nodes():
        if temporal_graph_dict[t][0].nodes[node]['observed'] == False and temporal_graph_dict[t][0].nodes[node]['state'] == 'I':
            if temporal_graph_dict[t][0].nodes[node]['infected_time'] >= self_report_days:
                print('self reporting')
                temporal_graph_dict[t][0].nodes[node]['observed'] = True
                temporal_graph_dict[t][0].nodes[node]['SIR_prob'] = [0,0,1]
                temporal_graph_dict[t][0].nodes[node]['state'] = 'R'
                BackloopUpdate(temporal_graph_dict, t, delta, node, 'I')

In [97]:
#mode = 'MF', 'R', 'None'
node_count = 20
infection_rate = 0.05
recovery_rate = 0
contact_radius = 2
time_steps = 80
temporal_graph_dict = {}
observe_per_day = 0
t_mf = 15
t_ct = 5
delay = 10
mode = 'CT'
initial_infection = 2
#infected person will report themselves after n days and will quarantine themselves(put into removed)
self_report = 5
self_report_mode = True
color_map = {'S': 'green', 'I': 'red', 'R': 'blue'}
def sim(DataFrame, id, mode, queue = None, self_report_mode = False):
    temporal_graph_dict = generate_graphs(node_count, contact_radius, time_steps)
    infectRandomNode(temporal_graph_dict[0][0], initial_infection)
    graph = temporal_graph_dict[0][0]
    S, I, R = graph.graph['S_count'], graph.graph['I_count'], graph.graph['R_count']
    DataFrame.loc[len(DataFrame)] = [id, 0, S, I, R]
    #string for description of loop:
    desc = 'Simulating (mode: '+mode +')'
    for i in tqdm(range(1, time_steps), desc=desc):
        graph = update(temporal_graph_dict[i-1][0], temporal_graph_dict[i][0], infection_rate, recovery_rate)
        #people will report themselves after n days if they are infected
        if self_report_mode == True:
            self_reporting(temporal_graph_dict, i, t_ct, self_report)
        if (mode == 'MF' and i > delay):
            pred = MeanFieldInference(graph, infection_rate, recovery_rate)
            sort_pred = sorted(pred.items(), key=lambda x: x[1])
            node_list = [node[0] for node in sort_pred]
            #print("day", i)
            #print(pred)
            #print(node_list)
            list_remove = DetectObserve(node_list, temporal_graph_dict,observe_per_day,i, t_mf)
            #print(list_remove)
            #set the nodes to be observed and put into removed in the current graph
            SetRemove(temporal_graph_dict[i][0], list_remove)
            #make sure i-t_mf is not negative
            lower_bound = max(0, i-t_mf)
            for time in range(lower_bound, i):
                pred = MeanFieldInference(temporal_graph_dict[time][0], infection_rate, recovery_rate)
                for node in temporal_graph_dict[time][0].nodes():
                    if temporal_graph_dict[time][0].nodes[node]['observed'] == False:
                        temporal_graph_dict[time][0].nodes[node]['SIR_prob'] = pred[node]
        elif (mode == 'R' and i > delay):
            #print("dayR", i)
            sample = Sampler(temporal_graph_dict[i][0], observe_per_day)
            SetRemove(temporal_graph_dict[i][0], sample)
        elif (mode == 'CT' and i > delay):
            ranker = {}
            for node in temporal_graph_dict[i][0].nodes():
                if temporal_graph_dict[i][0].nodes[node]['observed'] == False:
                    ranker[node] = contactTrace(temporal_graph_dict, i, node, t_ct)
            sort_ranker = sorted(ranker.items(), key=lambda x: x[1])
            node_list = [node[0] for node in sort_ranker]
            list_remove = DetectObserve(node_list, temporal_graph_dict,observe_per_day,i, t_ct)
            SetRemove(temporal_graph_dict[i][0], list_remove)
        S, I, R = graph.graph['S_count'], graph.graph['I_count'], graph.graph['R_count']
        DataFrame.loc[len(DataFrame)] = [id, i, S, I, R]
    if queue != None:
        queue.put(DataFrame)
    return temporal_graph_dict

In [98]:
Dataframe = pd.DataFrame(columns=['id', 'time', 'S', 'I', 'R'])
tempograph = sim(Dataframe, 1, mode, self_report_mode=True)

Generating Graphs:   0%|          | 0/80 [00:00<?, ?it/s]

C:\Users\Frank\AppData\Local\Temp\ipykernel_6368\1360817927.py:3: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  sample = random.sample(G.nodes(), G.number_of_nodes())


Simulating (mode: CT):   0%|          | 0/79 [00:00<?, ?it/s]

self reporting
self reporting
self reporting
self reporting
self reporting
self reporting
self reporting
self reporting
self reporting
self reporting
self reporting
self reporting
self reporting
self reporting


In [58]:
def drawgraph(tempograph, t):
    graph = tempograph[t][0]
    custom_pos = tempograph[t][1]
    colors = [color_map[graph.nodes[node]['state']] for node in graph.nodes()]
    nx.draw(graph, pos = custom_pos, node_color = colors, with_labels = True)

In [99]:
from ipywidgets import *
interact(drawgraph, tempograph = fixed(tempograph), t = widgets.IntSlider(min=0, max=time_steps-1, step=1, value=0))

interactive(children=(IntSlider(value=0, description='t', max=79), Output()), _dom_classes=('widget-interact',…

<function __main__.drawgraph(tempograph, t)>

In [18]:
#parrallelize simulation per 20 threads
number_of_sim = 100
thread_sim = 20
thread_list = []
Dataframe_final = pd.DataFrame(columns=['id', 'time', 'S', 'I', 'R'])
Queu_sim = queue.Queue()
for x in tqdm(range(number_of_sim), desc='Simulating Main (mode: '+mode +')'):
    Dataframe = pd.DataFrame(columns=['id', 'time', 'S', 'I', 'R'])
    thread = threading.Thread(target=sim, args=(Dataframe, x, mode, Queu_sim), name='thread' + str(x))
    thread.start()
    thread_list.append(thread)
    if len(thread_list) >= thread_sim:
        #reduce thread list
        thread = thread_list.pop(0)
        thread.join()
for thread in tqdm(thread_list, desc='Reducing thread Simulating (mode: '+mode +')'):
    thread.join()
while not Queu_sim.empty():
    Dataframe = Queu_sim.get()
    Dataframe_final = Dataframe_final.append(Dataframe)

Simulating Main (mode: CT):   0%|          | 0/100 [00:00<?, ?it/s]

Generating Graphs:   0%|          | 0/80 [01:12<?, ?it/s]

C:\Users\Frank\AppData\Local\Temp\ipykernel_31324\3500474817.py:3: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  sample = random.sample(G.nodes(), G.number_of_nodes())


Generating Graphs:   0%|          | 0/80 [00:04<?, ?it/s]

Generating Graphs:   0%|          | 0/80 [00:21<?, ?it/s]

...

Simulating (mode: CT):   0%|          | 0/79 [01:20<?, ?it/s]

Simulating (mode: CT):   0%|          | 0/79 [01:21<?, ?it/s]

Simulating (mode: CT):   0%|          | 0/79 [01:24<?, ?it/s]

Simulating (mode: CT):   0%|          | 0/79 [01:26<?, ?it/s]

C:\Users\Frank\AppData\Local\Temp\ipykernel_31324\1105863710.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Dataframe_final = Dataframe_final.append(Dataframe)
C:\Users\Frank\AppData\Local\Temp\ipykernel_31324\1105863710.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Dataframe_final = Dataframe_final.append(Dataframe)
C:\Users\Frank\AppData\Local\Temp\ipykernel_31324\1105863710.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Dataframe_final = Dataframe_final.append(Dataframe)
C:\Users\Frank\AppData\Local\Temp\ipykernel_31324\1105863710.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Dataframe_final = Dataframe_fina

In [ ]:
csv_desc = "Data" + mode + "_" + str(node_count) + "_" + str(infection_rate) + "_" + str(recovery_rate) + "_" + str(contact_radius) + "_" + str(time_steps) + "_" + str(observe_per_day) + "_" + str(delay) + "geo.csv"
target_dir = "./data/geoTMF15/"
Dataframe_final.to_csv(target_dir + csv_desc, index=False)